In [ ]:
!pip install --upgrade astrapy pandas

from astrapy import DataAPIClient
import pandas as pd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.5/300.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [ ]:
from astrapy import DataAPIClient

# Initialize the client
client = DataAPIClient("AstraCS:ycCqpHAvXZnAUOrRQZQdkdMw:98132933d955132d8824c1f208242e5a90362f299ee97e34c9d620fb531bd948")
db = client.get_database_by_api_endpoint(
  "https://47bfa723-13db-41da-a8e1-3232e7c367f2-us-east-2.apps.astra.datastax.com"
)

print(f"Connected to Astra DB: {db.list_collection_names()}")

Connected to Astra DB: ['sales_bronze']


In [ ]:

bronze = db.get_collection("sales_bronze")
silver = db.create_collection("sales_silver")

In [ ]:
data = bronze.find()
df = pd.DataFrame(data)
df.head()


,_id,order_id,region,country,item_type,sales_channel,order_priority,order_date,ship_date,units_sold,unit_price,unit_cost,total_revenue,total_cost,total_profit
0,19df6968-2861-4f8d-9f69-682861ff8d31,667593514,Middle East and North Africa,Morocco,Clothes,Online,M,9/14/2013,10/19/2013,4611,109.28,35.84,503890.08,165258.24,338631.84
1,a0e6a158-2289-40cb-a6a1-58228930cba5,752525556,Europe,Albania,Baby Food,Offline,H,7/19/2012,8/13/2012,7890,255.28,159.42,2014159.20,1257823.80,756335.40
2,252d400e-90fb-41e3-ad40-0e90fb21e33d,156530129,Sub-Saharan Africa,Tanzania,Fruits,Offline,H,10/20/2014,11/29/2014,9599,9.33,6.92,89558.67,66425.08,23133.59
3,769a5bf0-6aa7-4fe3-9a5b-f06aa72fe3c0,156295812,Australia and Oceania,East Timor,Cereal,Offline,M,12/7/2015,12/30/2015,259,205.70,117.11,53276.30,30331.49,22944.81
4,049fa0ee-e33e-4bc1-9fa0-eee33e8bc1fe,324669444,Europe,France,Cosmetics,Online,M,12/7/2015,1/18/2016,5758,437.20,263.33,2517397.60,1516254.14,1001143.46


In [ ]:
df.columns = df.columns.str.strip().str.lower()


In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')
df['ship_date'] = pd.to_datetime(df['ship_date'], errors='coerce')


In [ ]:
numeric_cols = ['units_sold', 'unit_price', 'unit_cost', 'total_revenue', 'total_cost', 'total_profit']

for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')


In [ ]:
df.dropna(inplace=True)


In [ ]:
df['order_date'] = df['order_date'].dt.tz_localize('UTC')
df['ship_date'] = df['ship_date'].dt.tz_localize('UTC')


In [ ]:
for _, row in df.iterrows():
    data = row.to_dict()
    silver.insert_one(data)
